In [1]:
import threading
import serial
import tkinter as tk
from datetime import datetime
import numpy as np

In [2]:
# Create an output folder (here OUTPUT-FOLDER) and adapt this directory
directory = "C:/Users/User/Dropbox/OUTPUT-FOLDER"

In [3]:
reading_Arduino = True
lines_per_file = 100
file_name = "RH_temp_records_"

# Adapt the COM where Arduino board is connected
ser = serial.Serial('COM4', 9600, timeout=1)

In [4]:
class MyGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.create_window()
        self.add_widgets()
        self.text_box = tk.Text(self.root, height = 2, width = 850)
        self.text_box.pack()
        return
    def create_window(self):
        self.root.geometry("900x100")
        self.root.title("RH and T monitoring")
        self.root.configure(background='white')
        return
    def Read_Arduino_line(self):
        line =  ser.readline()
        string = line.decode("utf-8")
        arduino_line = string.strip()
        now = datetime.now()
        now_str = now.strftime("%m/%d, %H:%M:%S")
        self.Print_text(now_str+" "+arduino_line)
        arduino_line = str(now)+" --> "+arduino_line
        return arduino_line

    def Arduino2txt(self,array_lines,file):
        for line in array_lines:
            with open(file, 'a') as the_file:
                the_file.write(line+'\n')
        return
    def Print_text(self,text=" "):
        self.text_box.delete("1.0","end")
        self.text_box.insert(tk.END, text)
        return
    def on_closing(self):
        if(tk.messagebox.askyesno(title="Quit?", message="Confirm quit?")):
            self.root.destroy()
        return
    def Get_file_index(self,counter):
        f = str(datetime.now())
        file_index = f.split(" ")[0]
        return file_index+str(counter)
    def Thread_monitor(self):
        t1 = threading.Thread(target=self.Monitor_RH)
        t1.start()
        return
    def add_widgets(self):
        button = tk.Button(self.root, text="START Monitoring" ,
                           font=("Arial",18),
                           command=self.Thread_monitor,
                           bg='white')
        button.pack(padx=10,pady=10)
    def Monitor_RH(self):
        counter = 0
        i = 0
        file_index = self.Get_file_index(counter)
        array_lines = []
        while(reading_Arduino):
            i = i+1
            read_line = self.Read_Arduino_line()
            array_lines = np.append(array_lines, read_line)
            if(i > lines_per_file):
                self.Arduino2txt(array_lines, directory+'/'+file_name+file_index+'.txt')
                i = 0
                file_index = self.Get_file_index(counter)
                counter = counter +1
                array_lines = []

In [ ]:
app = MyGUI()
app.root.mainloop()